# Example: Deep structural causal model counterfactuals

In [ ]:
from typing import Dict, List, Optional, Tuple, Union, TypeVar

import torch
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample, PyroParam
from pyro.contrib.autoname import scope
from pyro.poutine import condition, reparam

import causal_pyro
from causal_pyro.query.do_messenger import do
from causal_pyro.counterfactual.handlers import Factual, MultiWorldCounterfactual, TwinWorldCounterfactual

## Background: Normalizing flows and counterfactuals

Much of the causal inference literature has focused on relatively simple
causal models with low dimensional data. In order to perform
counterfactual reasoning in more complex domains with high dimensional
data, Palowski et al. [@pawlowski2020deep] introduced *deep structural
causal models* (Deep SCMs): SCMs with neural networks as the functional
mechanisms between variables.

Specifically, the neural networks are
*normalizing flows*. A normalizing flow transforms a base probability
distribution (often a simple distribution, such as a multivariate
Gaussian) through a sequence of invertible transformations into a more
complex distribution (such as a distribution over images). When used
within a Deep SCM, the flow's base distribution is an exogenous noise
variable, and its output is an endogenous variable.

A salient property
of normalizing flows is that computing the likelihood of data can be
done both exactly and efficiently, and hence training a flow to model a
data distribution through maximum likelihood is straightforward. In
addition, the inverse of a normalizing flow can also typically be
efficiently computed, which renders the abduction step of a
counterfactual---inferring the posterior over exogenous variables given
evidence---trivial.

## Model: deep structural causal model

The following code models morphological transformations of MNIST,
defining a causal generative model over digits that contains endogenous
variables to control the width $t$ and intensity $i$ of the stroke:

In [ ]:
class DeepSCM(PyroModule):

    def forward(self):
        U = pyro.sample("U", dist.Normal(0, 1))
        T = pyro.sample("T", ...)
        I = pyro.sample("I", ...)  # I(T)
        X = self.f_X(U, I, T)
        return X

## Query: counterfactual data generation

Next we ask a *counterfactual* question: given an observed digit $X$, what
would the digit have been had $t$ been $t + 1$?

To compute this quantity we would normally:
   1. invert the model to find latent exogenous noise $u$
   2. construct an intervened model
   3. re-simulate the forward model on the $u$ [@pearl2011algorithmization].  

However, we can equivalently
represent this process with inference in a single, expanded
probabilistic program containing two copies of every deterministic
statement (a so-called \"twin network\" representation of
counterfactuals, first described in Chapter 7 of [@pearl] and extended
to the PPL setting in [@tavares_2020])

In [ ]:
def deep_scm_query(model: DeepSCM):
    return do(actions={"T": 1})(
        condition(data={"X": x_obs})(
            TwinWorldCounterfactual(dim=-1)(
                reparam(config=pyro.infer.reparam.AutoReparam())(
                    model))))

cf_model = deep_scm_query(DeepSCM())

Like all counterfactuals, this estimand is not identified in general
without further assumptions: learning parameters $\theta$ that match
observed data does not guarantee that the counterfactual distribution
will match that of the true causal model. 

However, as discussed in the
original paper [@pawlowski2020deep] in the context of modeling MRI
images, there are a number of valid practical reasons one might wish to
compute it anyway, such as explanation or expert evaluation.

## Appendix: Inductive Biases for Identifiable Counterfactuals

Consider two types of counterfactuals; one of the form *\"Given $X$ was
observed as $x$, what would $Y$ have been had $X$ been $x'$\"* and
another of the form *\"Given $X$ was observed as $x$ and $Y$ was
observed as $y$, what would $Y$ have been had $X$ been $x'$?\"*.

The former type is potentially identifiable without an SCM using
observational or interventional data [@pearl; @richardson2013single].
*Effect of treatment on the treated* is a useful example of the former
that is widely useful.

The second type of counterfactual is called a
"twin-world counterfactual\" because it predicts $Y$ in a world where we
*do* $X=x'$ conditional on information from a world where $X=x$ already
caused $Y=y$ [@pearl].

Structural counterfactuals enable interesting
counterfactual quantities such as probability of necessity and
sufficiency, and have interesting applications in generative
explanations as well as quantifying regret, blame, and responsibility in
decision theory and agent modeling.

However, inferring twin-world counterfactual counterfactuals require an explicit SCM.
Further, if the SCM is misspecified, it can produce incorrect counterfactual inferences
even if it is a perfect statistical fit for observational and
interventional data.

How do we do select the "right\" surrogate SCM?

### Reparameterization Tricks

A tempting approach is to simply convert a directed generative model
into an SCM using "reparameterization tricks\"
[@kingma2015variational; @jang2016categorical], methods that shunt
randomness to *exogenous* variables to facilitate back-propagation
through *endogenous* variables.

The problem is that in general a causal generative model can yield
different SCMs depending on how it is reparameterized, and the different
SCMs might yield different counterfactual inferences. To illustrate,
consider the following causal generative model.

    def cgmodel():
      $x_1 \sim$ Bernoulli(0.5)
      $x_2 \sim$ Bernoulli(0.5)
      $y \sim$ Categorical$(p_{x_1, x_2}=g(x_1, x_2))$

Suppose we wished to "reparameterize\" this into an SCM. To accomplish
this, we shunt the randomness in the Bernoulli and categorical
distributions to *exogenous* variables $N_{X_1}$, $N_{X_2}$, and $N_{Y}$
through deterministic transformations $f_{X_1}$, $f_{X_2}$, and $f_{Y}$.

    def true_dgp():
      $n_{x_1} \sim \text{Bernoulli}(0.5)$
      $n_{x_2} \sim \text{Bernoulli}(0.5)$
      $n_y \sim \text{UniformDiscrete}([0, 1, 2])$
      $x_1 = f_{X_1}(n_{x_1}})$
      $x_2 = f_{X_2}(n_{x_2}})$
      $y = f_{Y}(x_1, x_2, n_y)$

In this case, $f_{Y,a}$ and $f_{Y,b}$ are two different alternatives for
$f_Y$ above that would each yield the same observational and
interventional distributions:

    def $f_{Y,a}(x_1, x_2, n_y)$:
        if ($x_1$ != $x_2$):
            if($n_y$ == 0):
                return $x_1$
            else:
                return $x_2$
        else:
            return $n_y$

    def $f_{Y,b}(x_1, x_2, n_y)$:
        if ($x_1$ != $x_2$):
            if($n_y$ == 0):
                return $x_1$
            else:
                return $x_2$
        else:
            return $2-n_y$

However, they would produce different counterfactual inferences. Suppose
we conditioned scmmodel on the observation
$\{X_1 = 1, X_2 = 0, Y = 0 \}$ and we are interested in the
counterfactual query "what would $Y$ have been if $X_1$ had been 0?" In
this degenerate case, we would infer a point value $N_Y = 0$. When we
re-execute the model after setting both $N_Y$ and $X_1$ to 0, $f_{Y,a}$
would yield 0, and $f_{Y,b}$ would yield 2.

### Monotonicity as Inductive Bias {#sec:monotonicity}

One solution is to limit ourselves to reparameterizations where key
counterfactual queries are identifiable from observational and
interventional data. [@pearl] named this constraint *monotonicity* and
defined it for the binary outcome case. [@oberst2019counterfactual]
extended the definition to categorical variables and showed that the
Gumbel-softmax reparameterization trick ([@jang2016categorical])
produced a monotonic SCM. [@ness2019integrating] extended the definition
to binomial and Poisson outcomes and provided a probabilistic
programming implementation.

In machine learning, we often talk about the inductive bias of a model,
such as how convolutional neural networks with max-pooling favor
translation invariance. In contrast to inductive biases implicit in
architecture, a strength of the probabilistic programming community is
that we favor explicit inductive biases, i.e. constraining inference
with domain knowledge built into the model.

Monotonicity is an example of an explicit inductive bias. To illustrate,
suppose Anne has the flu but still goes to work. Jon is exposed to Anne
($X_1 = 1$) and and a few days later, Jon got the flu ($Y = 1)$. Jon is
may or may not have had exposure to the flu on the bus ($X_2$), which is
unknown. Given knowledge that Jon was exposed to Anne and he got the
flu, what are the chances he wouldn't have gotten the flu if Anne had
stayed home ($P(Y_{X_1=0}=0|X_1=1, Y=1)$)?

Given sufficient data, we could build a good probabilistic model of
$P(X_1, X_2, Y)$. Theoretically we know that if we were to apply a
monotonic reparameterization (specifically with respect to $X_1$ and
$Y$) to an SCM then we could use that model to infer the above
counterfactual. How would we know if monotonicity is a valid
counterfactual inductive bias in this case?

We can answer with a simple thought experiment. Is it conceivable that
some strange group of coworkers could have the flu, but then *be cured
by* exposure to Anne? That would be a case of non-monotonicity. In this
case that is implausible, and thus monotonicity is a safe assumption.

Suppose, however, that $X_1$ were email promotion and $Y$ were sales,
and we were interested in if John would have bought a product if he
hadn't seen an email promotion. In our thought experiment we would ask
it is plausible for some people who intended to buy a product to be
annoyed enough by an email promotion that they then decided not to buy.
In that case, monotonicity would not be a safe assumption, since
non-monotonicity is plausible.

